<a href="https://colab.research.google.com/github/vafter341ew/COLAB/blob/main/lab6_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import files
import zipfile
import shutil

print("Upload your zip file:")
uploaded = files.upload()

zip_filename = list(uploaded.keys())[0]
with zipfile.ZipFile(zip_filename, 'r') as z:
    z.extractall()


Upload your zip file:


Saving Lab 6.1 - Summarizing the Heathcare Survey with Pyspark attached files Nov 25, 2025 953 PM.zip to Lab 6.1 - Summarizing the Heathcare Survey with Pyspark attached files Nov 25, 2025 953 PM (2).zip


In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, mean

spark = SparkSession.builder.appName("HealthSurvey").getOrCreate()

reverse_items = spark.read.csv("ReverseCodingItems_v2.csv", header=True, inferSchema=True)
reverse_list = [row["Column Name"] for row in reverse_items.filter(col("Needs Reverse Coding?") == "Yes").collect()]

health_cols = [c for c in spark.read.csv("health_survey_v2.csv", header=True).columns if c != "ID"]

df = spark.read.csv("health_survey_v2.csv", header=True, inferSchema=True)

stacked = df.selectExpr("ID", f"stack({len(health_cols)}, " + ", ".join([f"'{q}', `{q}`" for q in health_cols]) + ") as (Question, Response)")

joined = stacked.join(reverse_items.select(col("Column Name"), col("Construct")), stacked.Question == reverse_items["Column Name"])

recoded = joined.withColumn("Recoded Value",
    when(col("Question").isin(reverse_list),
        when(col("Response") == "Strongly Disagree", 5)
        .when(col("Response") == "Somewhat Disagree", 4)
        .when(col("Response") == "Neither Agree nor Disagree", 3)
        .when(col("Response") == "Somewhat Agree", 2)
        .when(col("Response") == "Strongly Agree", 1))
    .otherwise(
        when(col("Response") == "Strongly Disagree", 1)
        .when(col("Response") == "Somewhat Disagree", 2)
        .when(col("Response") == "Neither Agree nor Disagree", 3)
        .when(col("Response") == "Somewhat Agree", 4)
        .when(col("Response") == "Strongly Agree", 5)))

factored = recoded.withColumn("Factor",
    when(col("Construct") == "1", "F1")
    .when(col("Construct") == "2", "F2")
    .when(col("Construct") == "3", "F3")
    .when(col("Construct") == "4", "F4")
    .when(col("Construct") == "5", "F5")
    .when(col("Construct") == "6", "F6"))

result = factored.groupBy("ID", "Factor").agg(mean("Recoded Value").alias("Mean Score")).groupBy("ID").pivot("Factor").agg(mean("Mean Score"))

result.write.csv("health_survey_summary.csv", header=True, mode="overwrite")

spark.stop()